In [1]:
pip install streamlit run

Defaulting to user installation because normal site-packages is not writeable
  Using cached run-0.2.tar.gz (3.2 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(c

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('song.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...


In [4]:
df.shape

(57650, 4)

In [5]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
df.shape

(5000, 3)

In [7]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [8]:
df['text'][0]

"chorus x2:  \r who you gonna call when it's time to brawl  \r standin round waitin for my queendom to fall  \r well i think not, styles are pipin hot  \r blazin, amazin, i give it all i got  \r   \r i'm tellin you straight up, all sleepin beauties better wake up  \r ill tear your state up, so set the date up  \r and i'm a rip it, what if it what was it  \r who did it, who does it  \r from private to public  \r anywhere i'm in there and been there  \r so recognize this, who the nicest  \r sit down and settle for your constellation prizes  \r whatever you want i got  \r whether you ready or not  \r its about to get hot when i drop  \r so notes, pause another spot  \r to do you, don't get me in a corner make me do you  \r don't try to be me, do you  \r be coo to you and do you  \r i'm on a higher level with different class, another plane  \r i am the queen, that's my name, time to explain  \r that i spit game with dames  \r leave em all with shit stains  \r split frames, hopin you hopin 

In [9]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [10]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [13]:
similarity[0]

array([1.        , 0.0351612 , 0.01160701, ..., 0.0530302 , 0.05297077,
       0.07016724])

In [14]:
df[df['song']=='here we go, here we go now ']

,artist,song,text


In [15]:
print(df.head())


          artist            song  \
0  Queen Latifah         Bananas   
1   Eric Clapton     Little Wing   
2  Planetshakers  Come To Praise   
3      Helloween  Revolution Now   
4    Linkin Park   Easier To Run   

                                                text  
0  choru x2 : who you gon na call when it 's time...  
1  well she 's walk through the cloud with a circ...  
2  when i think of the way that he gave all for m...  
3  well , i think it 's realli time again for a r...  
4  it 's easier to run replac thi pain with somet...  


# recommedation function

In [16]:
def recommendation(song_df):
    idx = df[df['song'] == song_df]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [17]:
recommendation('here we go, here we go now')

[]

In [18]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))